In [24]:
#A code to analyze image cubes from the JWST nirspec ifu - examples here are done with CO
#By Adam E. Rubinstein

# load important packages
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d #UnivariateSpline
import os
import sys

from astropy.io import fits #, ascii
from astropy import units as u
import pandas as pd
from spectral_cube import SpectralCube

# Read in CO data

Also can show line/continuum ratio as an optional figure idea

In [25]:
#processing fits images of CO line emission to be saved...
#first define some lists to loop over for the different CO line series
# protostar_list = ['IRAS16253', 'B335', 'HOPS153', 'HOPS370', 'IRAS20126']
# protostar_names = ['IRAS 16253', 'B335', 'HOPS 153', 'HOPS 370', 'IRAS 20126'] #just to add some spaces
v_list = ['1*'] #, '2*', '3*']
j_list = ['R*', 'P*']
pc_to_cm = 3.086e18
dist_list = pc_to_cm*np.array([150, 165, 390, 390, 1.55e3]) #pc: from Ortiz-Leon 2018, Watson 2020, Tobin+2020a, Tobin+2020a, Reid+2019
protostars = ['IRAS16253', 'B335', 'HOPS153', 'HOPS370', 'IRAS20126']

#some lists related to correcting for ices
transmission_folder = '../Ices/'
transmission_file_end = '_NIRspec_cube_pspline_asls_transmission.fits' 
offset_list = [1.85e-3, 2e-3, 1.1e-3, 1.1e-3, 2.25e-3] #experimental, from cube fitting, needed to fix wavelengths we're using here

#a function to generate lists of co line intensity maps for our protostars
def co_data_output(protostar_str, v_str, j_str):
    #read in some protostellar images for each protostar
    co_file_list = sorted(glob('../Line_Images/'+ protostar_str + '*_CO*v=' + v_str + j_str + '.fits'), key=lambda x: int(x.split('_')[-2][-2:]), reverse=True) #change the wildcard '*' here!; note we need the key to order images by number...

    #extract useful values and labels from file names in case of saving...
    #just using an example star to figure out general process (whatever is first), then loop later
    co_j =  [i.split('_')[-2][-2:] for i in co_file_list] 
    co_wavelengths = np.array([float(i.split('_')[-1][:-5]) for i in co_file_list])

    #setting up and correcting units
    hdul_co = [fits.open(j) for j in co_file_list]     #extract hdu files from paths
    sr_conversion = np.abs(hdul_co[0][0].header['CDELT1'] * hdul_co[0][0].header['CDELT2'] / (180. / np.pi)**2.0) #square degrees to steradians, should be consistent throughout all images (?)
    # mega_conversion = 1e6 #can swap out with Jy conversion to preferred units...
    co_data = [j[0].data*sr_conversion for j in hdul_co]     #extracting data for each hdu, converting units

    #reading in transmission file needed to slightly correct data
    protostar_ind = protostars.index(protostar_str) #if you'd like to do this by indexing
    hdul_transmission = fits.open(transmission_folder + protostar_str + transmission_file_end) #read in hdu
    transmission_cube = SpectralCube.read(hdul_transmission[0]) #accessing the cube for data  
    transmission_data = np.array(transmission_cube._data, dtype=np.float64) #open transmission slices
    co_header = transmission_cube[0].header #we need headers for later, but we can only take one slice (doesn't work if NAXIS > 2: https://github.com/aplpy/aplpy/issues/449)
    jwst_wavelengths = transmission_cube.spectral_axis.value/1e-6 + offset_list[protostar_ind]
    transmit_splined = interp1d(jwst_wavelengths, transmission_data, axis=0) #transmission data

    #correcting for ices by dividing them out as a transmission filter centered at the peak wavelength of each line
    # wave_cutoff_idx = np.where((co_wavelengths > 4.52)*(co_wavelengths < 4.96))[0]     #first spline the transmission curve, masking it where we want to apply it (wavelengths between 4.52 and 4.96)
    # co_data_corrected = [co_data[j] / transmit_splined_list[protostar_ind](co_wavelengths[j]) for j in wave_cutoff_idx] #then apply transmission curve
    co_data_corrected = []
    for i in range(len(co_wavelengths)): #making a brute force loop and condition since the mask wasn't working right
        if co_wavelengths[i] > 4.52 and co_wavelengths[i] < 4.96:
            co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
        else:
            co_data_corrected.append(co_data[i])

    #applying smoothing or SNR cut (optional)
    # jwst_err_cube = hdul['ERR'].data #not using spectral cube here
    # err_flux1 = np.array(jwst_err_cube[:, ind1, ind2], dtype=np.float64)
    # flux1[ flux1/err_flux1 < 1] = 0
    # print(type(flux1[0])) #checking if float64

    return co_j, co_wavelengths, co_data, co_data_corrected, co_header, transmit_splined

# Collect lab values
Like Einstein coefficients

In [26]:
#reading in from line list for CO
#done first so we can add it to our long table!
# columns = A, sec-1	E_u/k	g_u
table_path = '../Line list 2.1 for python.xlsx'
column_names = ['Wv, microns', 'A, sec-1', 'E_u/k', 'g_u']
co_lab_props = pd.read_excel(table_path, sheet_name='CO', usecols=column_names)

# Compute extinction, tau across map

In [36]:
#optically thin case for foreground extinction
from scipy.interpolate import CubicSpline

#for de-extincting, you need the file below! It is taken from Bruce Draine's website for Rv = 5.5, to be interpolated
#https://ui.adsabs.harvard.edu/abs/2001ApJ...548..296W/abstract
#this is used for the cross sections Cext or kext, which can be used to compute tau or Av
#extracting data
# kext_dat = np.genfromtxt('kext_albedo_WD_MW_5.5A_30_D03_delimmod.txt', delimiter=' ', dtype=None, skip_header=80, usecols=range(0,5)) #reading in excel file of extinctions
# kext_dat_ascend = sorted(kext_dat, key=lambda x:x[0]) #sorted in ascending order, from small to large lambda
kext_dat_ascend = pd.read_excel('../extinc_kp5.0_rv_ncol.xlsx') #UPGRADE: klaus pontoppidan 5.0

#extracting relevant data and interpolating
lam_kext = kext_dat_ascend['lambda, microns'].values  # [i[0] for i in kext_dat_ascend]
Cext_kext = kext_dat_ascend['extinction cross section, cm^2'].values # [i[4] for i in kext_dat_ascend] #cross section per H nucleon (?); change columns depending on file
vs = CubicSpline(lam_kext, Cext_kext) #the all important interpolating function, note it comes from Cext on the table

#function and global lambdaV for reference
lambdaV = 0.547
def extinc(co_r_wavelengths, co_p_wavelengths, co_r_p_ratio_list, co_A_ij_r, co_A_ij_p):
    #computing intrinsic lab measured ratio
    r_to_p_0 = co_A_ij_r/co_A_ij_p * co_p_wavelengths/co_r_wavelengths #from HITRAN
    C_ext_co = vs(co_r_wavelengths) - vs(co_p_wavelengths) #needed to define "Rv" for our set of CO transitions
    
    #computing tau for each image, then taking image * exp(tau) to de-extinct, and saving
    ''' 
    av = vs(lambdaV)/C_ext_co * 2.5 * np.log10(r_to_p_0 / i)
    av_err = vs(lambdaV)/C_ext_co * 2.5 * r_to_p_0 / np.log(10.0) * co_err_r_p_ratio_list[i] / co_r_p_ratio_list[i]
    '''
    # co_red_list = 2.5 * np.log10(r_to_p_0 / co_r_p_ratio_list) #the reddening Rv / Av = E
    # co_red_err_list = 2.5 * r_to_p_0 / np.log(10.0) * co_err_r_p_ratio_list / co_r_p_ratio_list #note that log(x) -> dx/x for uncertainty (e.g. https://openbooks.library.umass.edu/p132-lab-manual/chapter/uncertainty-for-natural-logarithms/)
    # co_av_list = vs(lambdaV)/C_ext_co * co_red_list #visual extinction, Av
    # co_av_err_list = vs(lambdaV)/C_ext_co * co_red_err_list

    #now we are closer to full analysis, but first we need tau in the continuum to deextinct...
    ''' 
    keep in mind we are doing a ratio of co values...this means we are doing a ratio of different tau ultimately...
    deext ratio = (flux_co_r * exp(tau_r)) / (flux_co_p * exp(tau_p))
    deext sum = (flux_co_r * exp(tau_r)) + (flux_co_p * exp(tau_p))
    '''
    tau_r_ext_list = vs(co_r_wavelengths[:,None,None]) / C_ext_co[:,None,None] * np.log(r_to_p_0[:,None,None] / co_r_p_ratio_list)
    tau_p_ext_list = vs(co_p_wavelengths[:,None,None]) / C_ext_co[:,None,None] * np.log(r_to_p_0[:,None,None] / co_r_p_ratio_list)

    #computing uncertainties in tau
    # tau_r_err_list = vs(co_r_wavelengths[:,None,None]) / C_ext_co * r_to_p_0 * co_err_r_p_ratio_list /  co_r_p_ratio_list
    # tau_p_err_list = vs(co_p_wavelengths[:,None,None]) / C_ext_co * r_to_p_0 * co_err_r_p_ratio_list /  co_r_p_ratio_list


    #we also want tau / sigma = Sigma = n L, a column density
    column_density_r_list = tau_r_ext_list / vs(co_r_wavelengths[:,None,None])
    column_density_p_list = tau_p_ext_list / vs(co_p_wavelengths[:,None,None])

    return column_density_r_list, column_density_p_list, tau_r_ext_list, tau_p_ext_list

In [37]:
#read in a list of spectral cubes to extract wcs headers...
file_list = [glob('../' + i + '/*.fits')[0].replace('\\', '/') for i in protostars]
hdul = [fits.open(filepath) for filepath in file_list]
jwst_cube_list = [SpectralCube.read(hdu[1]) for hdu in hdul] #accessing the cube for data  
jwst_wcs_list = [cube.wcs for cube in jwst_cube_list]

In [39]:
from astropy.wcs import WCS

#input values, such as wavelength resolution for our lookup tables
wave_lookup_res = 0.000001
protostar_idx = 0 #for use in tests
avg_ext_arr = [] #empty list needed to be filled for each protostar
avg_av_unc_arr = [] #empty list needed to be filled for each protostar
# transmiss_threshold_list = [[0.64], [0.69, 0.69, 0.69], [0.64, 0.64], [0.84, 0.84, 0.84], [0.84, 0.84]]
transmiss_threshold_list = [0.62, 0.6, 0.63, 0.7, 0.7]

#loop thorugh protostars and compute a table of fluxes:
for i in range(len(protostars)):
    count = 0
    co_av_list = []
    av_masks = [] #empty list to be filled for each protostar

    #loop through vibrational states (each axis)
    for j in range(len(v_list)):
        #loop through the rotational states to collect a set of data and usable values
        #doing it by hand here since will only ever be two options
        co_v_r_j, co_v_r_wavelengths, co_v_r_data, co_v_r_data_corrected, co_header, transmit_splined = co_data_output(protostars[i], v_list[j], j_list[0])
        co_v_p_j, co_v_p_wavelengths, co_v_p_data, co_v_p_data_corrected, co_header, transmit_splined = co_data_output(protostars[i], v_list[j], j_list[1])

        #only can do for j with v = 1-0...
            #fix wavelengths and J
        co_v_r_sorted = np.array(co_v_r_j[8:][::-1])
        co_v_p_sorted = np.array(co_v_p_j[:-1][::-1])
        co_r_wave_sorted = np.array(co_v_r_wavelengths[8:][::-1])
        co_p_wave_sorted = np.array(co_v_p_wavelengths[:-1][::-1])
        co_r_flux_sorted = np.array(co_v_r_data[8:][::-1])
        co_p_flux_sorted = np.array(co_v_p_data[:-1][::-1])
        # co_r_fluxerr_sorted = np.array(co_err_r_flux_list[8:][::-1])
        # co_p_fluxerr_sorted = np.array(co_err_p_flux_list[:-1][::-1])

        if i == 2:
            print(co_v_r_sorted)
            print(co_v_p_sorted)

        #some useful ratios and propagated the ratio of uncertainties for later too
        co_r_p_ratio_list = co_r_flux_sorted / co_p_flux_sorted #note if conversion done here
        # co_err_r_p_ratio_list = co_r_p_ratio_list * np.sqrt( (co_r_fluxerr_sorted/co_r_flux_sorted)**2. + (co_p_fluxerr_sorted/co_p_flux_sorted)**2.) #note conversion done here

            
        #look up the co lab properties we need and split them into r and p branches for use in calculations
        #column_names[0] is 'Wv, microns', the first column of the co lab properties
        #wave_lookup_res is at the start of this code block and is about the minimum allowed
        #r-branch
        co_A_ij_r = [] #list of indices for use in looking up wavelengths and matching what we have to the lab properties
        for k in range(len(co_r_wave_sorted)):
            idx = (co_lab_props[column_names[0]] <= co_r_wave_sorted[k]+wave_lookup_res) & (co_lab_props[column_names[0]] > co_r_wave_sorted[k]-wave_lookup_res)
            co_A_ij_r.append(co_lab_props[column_names[1]][idx].values[0]) #apply indices to and lookup in table of lab properties
        co_A_ij_r = np.array(co_A_ij_r)

        #p-branch
        co_A_ij_p = [] #list of indices for use in looking up wavelengths and matching what we have to the lab properties
        for k in range(len(co_p_wave_sorted)):
            idx = (co_lab_props[column_names[0]] <= co_p_wave_sorted[k]+wave_lookup_res) & (co_lab_props[column_names[0]] > co_p_wave_sorted[k]-wave_lookup_res)
            co_A_ij_p.append(co_lab_props[column_names[1]][idx].values[0]) #apply indices to and lookup in table of lab properties
        co_A_ij_p = np.array(co_A_ij_p)

        #determine extinction here!
        # co_av_list, co_av_err_list, tau_r_ext_list, tau_p_ext_list, tau_r_err_list, tau_p_err_list = extinc(co_r_wave_sorted, co_p_wave_sorted, co_r_p_ratio_list, co_err_r_p_ratio_list, co_A_ij_r, co_A_ij_p)
        column_density_r_list, column_density_p_list, tau_r_ext_list, tau_p_ext_list = extinc(co_r_wave_sorted, co_p_wave_sorted, co_r_p_ratio_list, co_A_ij_r, co_A_ij_p)        
        avg_ext_arr.append(tau_r_ext_list)

        #saving extinction as cube, not summed
        # cube_unit_converted = np.multiply(cube_units, jwst_data)
        cube = SpectralCube(data=column_density_r_list, wcs=jwst_wcs_list[i])
        # cube = cube.with_spectral_unit(u.um)
        cube_savepath = 'CO_Extinction_Maps/'
        cube_name = protostars[i] + '_NIRspec_cube_CO_r_v' + str(j+1) + '_extcol.fits'
        cube.write(cube_savepath+cube_name, format='fits', overwrite=True)
        print('Saved: ', cube_savepath+cube_name)

        #saving baseline as cube, not summed, with MJy/sr
        # cube_unit_converted = np.multiply(cube_units, jwst_data)
        cube = SpectralCube(data=column_density_p_list, wcs=jwst_wcs_list[i])
        # cube = cube.with_spectral_unit(u.um)
        cube_savepath = 'CO_Extinction_Maps/'
        cube_name = protostars[i] + '_NIRspec_cube_CO_p_v' + str(j+1) + '_extcol.fits'
        cube.write(cube_savepath+cube_name, format='fits', overwrite=True)
        print('Saved: ', cube_savepath+cube_name)


#     #now to make some example plots; we will pick one for later
#     ### TRANSMISSION CURVE
#     #for each protostar, we can now define an averaged transmission function in order to threshold our extinction values
#     #must be done once for each vibrational state as well
#     transmiss_p_sorted = np.array([transmit_splined(k) for k in co_p_wave_sorted])
#     transmiss_r_sorted = np.array([transmit_splined(k) for k in co_r_wave_sorted])

#     ### AV HISTOGRAM + TESTING TRANSMISSION PERCENTAGE THRESHOLD
#     #need to filter out a given set of vibrational data here
#     for k1,k2 in zip(transmiss_r_sorted, transmiss_p_sorted):
#         if k1 > transmiss_threshold_list[i] and k2 > transmiss_threshold_list[i]:
#             # print(k1, k2, co_av_list[count])
#             av_masks.append(count)
#         count += 1
        
    # #compile values
    # av_arr = np.concatenate(av_arr)
    # av_masks = np.array(av_masks)
    # # print(av_masks)
    # # print(av_arr[av_masks])    
    
    # #masking values
    # av_pos_masked = av_arr[av_masks]
    # av_pos_masked[av_pos_masked < 0] = np.nan
    # av_pos_masked[av_pos_masked > 1000] = np.nan
    # avg_av_filtered = np.nanmedian(av_pos_masked)
    # avg_ext_arr.append(avg_av_filtered)
    # avg_av_unc_arr.append(np.nanstd(av_pos_masked, ddof=1)/np.sqrt(len(av_pos_masked)))
    # # print('Uncertainties: ', co_av_err_list)
    # print('Removed Outliers: ', avg_av_filtered, 'High Transmission, still outliers: ', np.nanmedian(av_arr[av_masks]))
    # print('StD of high transmission + outliers removed: ', np.nanstd(av_pos_masked, ddof=1)/np.sqrt(len(av_pos_masked)))

C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\285379221.py:40: RuntimeWarning: divide by zero encountered in true_divide
  co_r_p_ratio_list = co_r_flux_sorted / co_p_flux_sorted #note if convers

Saved:  CO_Extinction_Maps/IRAS16253_NIRspec_cube_CO_r_v1_extcol.fits
Saved:  CO_Extinction_Maps/IRAS16253_NIRspec_cube_CO_p_v1_extcol.fits


C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\285379221.py:40: RuntimeWarning: divide by zero encountered in true_divide
  co_r_p_ratio_list = co_r_flux_sorted / co_p_flux_sorted #note if convers

Saved:  CO_Extinction_Maps/B335_NIRspec_cube_CO_r_v1_extcol.fits
Saved:  CO_Extinction_Maps/B335_NIRspec_cube_CO_p_v1_extcol.fits


C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\285379221.py:40: RuntimeWarning: divide by zero encountered in true_divide
  co_r_p_ratio_list = co_r_flux_sorted / co_p_flux_sorted #note if convers

[' 0' ' 1' ' 2' ' 3' ' 4' ' 5' ' 6' ' 7' ' 8' ' 9' '10' '11' '12' '13'
 '14' '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27'
 '28' '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41'
 '42' '43' '44' '45' '46' '47' '48' '49' '50']
[' 2' ' 3' ' 4' ' 5' ' 6' ' 7' ' 8' ' 9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52']
Saved:  CO_Extinction_Maps/HOPS153_NIRspec_cube_CO_r_v1_extcol.fits
Saved:  CO_Extinction_Maps/HOPS153_NIRspec_cube_CO_p_v1_extcol.fits


C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\285379221.py:40: RuntimeWarning: divide by zero encountered in true_divide
  co_r_p_ratio_list = co_r_flux_sorted / co_p_flux_sorted #note if conversion done here
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\285379221.py:40: RuntimeWarning: invalid value encountered in true_divide
  co_r_p_ratio_list = co_r_flux_sorted / co_p_flux_sorted #note

Saved:  CO_Extinction_Maps/HOPS370_NIRspec_cube_CO_r_v1_extcol.fits
Saved:  CO_Extinction_Maps/HOPS370_NIRspec_cube_CO_p_v1_extcol.fits


C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: divide by zero encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\400570441.py:47: RuntimeWarning: invalid value encountered in true_divide
  co_data_corrected.append(co_data[i] / transmit_splined(co_wavelengths[i]))
C:\Users\arubi\AppData\Local\Temp\ipykernel_7880\285379221.py:40: RuntimeWarning: divide by zero encountered in true_divide
  co_r_p_ratio_list = co_r_flux_sorted / co_p_flux_sorted #note if convers

Saved:  CO_Extinction_Maps/IRAS20126_NIRspec_cube_CO_r_v1_extcol.fits
Saved:  CO_Extinction_Maps/IRAS20126_NIRspec_cube_CO_p_v1_extcol.fits
